In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
from Policies import policies
from IPython.display import Image
import matplotlib.pyplot as plt; from matplotlib.gridspec import GridSpec; from matplotlib.transforms import Affine2D
import scipy.stats as st
import imageio

# Auxiliary function to visualize episode
def print_state(env):
    print(f'################################### STEP {env.t} ###################################')
    print('INVENTORY')
    max_age = max(list(env.O_k.values()))
    string = 'M \ O \t'
    for o in range(1, max_age + 1):
        string += f' {o} \t \t'
    print(string)
    for k in env.Products:
        string = f'S{k} \t \t '
        for o in env.Ages[k]:
            string += f'{env.state[k,o]} \t'
        print(string)
    
    print('\n')
    print('DEMAND')
    string1 = 'K'
    string2 = 'd'
    for k in env.Products:
        string1 += f'\t {k}\t'
        string2 += f'\t {env.W_t["d"][k]}'
    print(string1)
    print(string2)

    print('\n')
    print('AVAILABLE QUANTITIES')
    string = 'M \ K \t'
    for k in env.Products:
        string += f'{k} \t \t'
    print(string)
    for i in env.Suppliers:
        new_string = f'{i} \t \t'
        for k in env.Products:
            if env.W_t['q'][i,k] == 0:
                new_string += f'{env.W_t["q"][i,k]} \t \t'
            else:
                new_string += f'{env.W_t["q"][i,k]} \t'
        print(new_string)


def print_extras(env, action, _):
    print('\n')
    print('PURCHASE')
    purchase = action[1]
    string = 'M \ K \t'
    for k in env.Products:
        string += f'{k} \t \t'
    print(string)
    for i in env.Suppliers:
        new_string = f'{i} \t \t'
        for k in env.Products:
            if purchase[i,k] == 0:
                new_string += f'{float(purchase[i,k])} \t'
            else:
                new_string += f'{round(purchase[i,k],2)} \t'
        print(new_string)
    
    print('\n')
    print('Demand compliance')
    demand_compliance = action[2]
    max_age = max(list(env.O_k.values()))
    string = 'K \ O \t'
    for o in range(max_age + 1):
        string += f' {o} \t \t'
    print(string)
    for k in env.Products:
        string = f'K{k} \t \t '
        for o in range(env.O_k[k] + 1):
            string += f'{round(demand_compliance[k,o],2)} \t'
        print(string)

    print('\n')
    print('BACKORDERS')
    string1 = 'K'
    string2 = 'bo'
    for k in env.Products:
        if k in _['backorders'].keys():
            string1 += f'\t {k}\t'
            string2 += f'\t {_["backorders"][k]}'
    print(string1)
    print(string2)

    print('\n \n \n \n \n')

# Policy Evaluation

This Notebook has a complete Policy Evaluation function for the Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products. First, the main parameters of the problem and the environment must be set. All the main customizable parameters are below.

In [5]:
#################################   Environment's parameters   #################################
# Random seed
rd_seed = 0

# SD-IB-IRP-PP model's parameters
backorderss = 'backorders'
stochastic_parameters = ['q','d']

# Feature's parameters
look_ahead = ['q','d']
historical_data = ['*']

# Action's parameters
validate_action = False
warnings = False

# Other parameters
num_episodes = 1
env_config = { 'M': 5, 'K': 5, 'T': 7,  'F': 4,
               'S': 10,  'LA_horizon': 3, 'back_o_cost': 1e5}

q_params = {'distribution': 'c_uniform', 'min': 6, 'max': 20}
d_params = {'distribution': 'log-normal', 'mean': 2, 'stdev': 0.5}

p_params = {'distribution': 'd_uniform', 'min': 1, 'max': 500}
h_params = {'distribution': 'd_uniform', 'min': 1, 'max': 500}

#################################   Environment's parameters   #################################

Next, the policy evaluation is defined. This function only takes one parameter, the number of episodes that will be runned of the environment. All the policies to be evaluated must be in the 'Policies.py' file. Any policy used must be able to receive the state, the additional information in _ and the environment as a parameters. Also, the policy function must return an action in the format defined in the Toying.ipynb.

In [6]:
'''

'''
def Policy_evaluation(num_episodes = 1000):
    
    # Episode's and performance storage
    rewards = {};   states = {};   real_actions = {};   backorders = {};   la_decisions = {}
    realized_dem = {};   q_sample = {};   tws = {}; perished = {}; realized_q = {}

    # Generating environment and policies generator
    env = steroid_IRP( look_ahead = look_ahead, 
                       historical_data = historical_data, 
                       backorders = backorderss,
                       stochastic_parameters = stochastic_parameters, 
                       env_config = env_config)

    policy = policies()

    for episode in range(num_episodes):

        state, _ = env.reset( return_state = True, rd_seed = rd_seed, q_params = q_params, 
                              p_params = p_params, d_params = d_params, h_params = h_params)

        
        done = False
        while not done:
            #print_state(env)
            # Environment transition
            states[episode,env.t] = state
            q_sample[episode,env.t] = [_["sample_paths"]["q"][0,s] for s in env.Samples]
            realized_dem[episode,env.t] = env.W_t["d"]
            realized_q[episode,env.t] = env.W_t["q"]
            action, la_dec = policy.Myopic_Heuristic(state, _, env)
            if done:    tws[episode,env.t] = 1
            else:    tws[episode,env.t] = _["sample_path_window_size"]
            state, reward, done, real_action, _,  = env.step(action, validate_action = validate_action, warnings = warnings)
            if done:    states[episode,env.t] = state
            print(env.t)
            #print_extras(env, real_action, _)

            # Data storage
            real_actions[episode,env.t-1] = real_action
            backorders[episode,env.t-1] = _["backorders"]
            perished[episode,env.t-1] = {k:_["perished"][k] if k in _["perished"] else 0 for k in env.Products}
            rewards[episode,env.t] = reward
            la_decisions[episode,env.t-1] = la_dec
            
            
    iterables = (env.Suppliers, env.Products, env.Samples, env.M_kt, env.O_k, env.Horizon)
    costs = (env.c, env.h_t, env.p_t, env.back_o_cost)

    return rewards, states, real_actions, backorders, la_decisions, realized_dem, q_sample, tws, iterables, costs, perished, realized_q

In [7]:
rewards, states, real_actions, backorders, la_decisions, realized_dem, q_sample, tws, iterables, costs, perished, realized_q = Policy_evaluation(num_episodes = num_episodes)

1
2
3
4
5
6
7


In [ ]:
def get_historical_decisions(episode, states, real_actions, backorders, K, T, O_k, perished):
    # Initial Inventory level for product k, aged o
    ii_0 = {t:states[episode,t] for t in range(len(T)+1)}
    # Final Inventory level
    ii_f = {t:{(k,o):ii_0[t+1][k,o+1] if o<O_k[k] else perished[episode,t][k] for k in K for o in range(O_k[k]+1)} for t in T}
    # Routing Decisions
    rout = {t:real_actions[episode,t][0] for t in T}
    # Purchasing decisions
    purch = {t:real_actions[episode,t][1] for t in T}
    # Backorders
    back = {t:{k:backorders[episode,t][k] if k in backorders[episode,t] else 0 for k in K} for t in T}

    return ii_0, ii_f, rout, purch, back

def get_lookahead_decisions(episode, day, la_decisions, T, K, O_k, S, I_0):
    # Final Inventory level
    ii_f_la = {(t):la_decisions[episode,day][0][t-day] for t in T}
    # Initial Inventory level
    ii_0_la = {(t):{s:{(k,o):ii_f_la[t-1][s][k,o-1] if t>T[0] else I_0[t][k,o] for k in K for o in range(1,O_k[k]+1)} for s in S} for t in T}
    # Purchase Decisions
    purch_la = {(t):la_decisions[episode,day][1][t-day] for t in T}
    # Backorders
    back_la = {(t):la_decisions[episode,day][2][t-day] for t in T}

    return ii_0_la, ii_f_la, purch_la, back_la

def charts_max_axis_values(episode, states, real_actions, backorders, la_decisions, tws, iterables, costs, perished):
    
    M, K, S, M_kt, O_k, T = iterables

    ''' Costs '''
    c = costs[0]
    h = costs[1]
    p = costs[2]
    g = costs[3]/1e3

    ''' Realized, Historical Decisions '''
    ii_0, ii_f, rout, purch, back = get_historical_decisions(episode,states,real_actions,backorders, K, T, O_k, perished)

    ''' Lookahead Decisions '''
    ii_0_la, ii_f_la, purch_la, back_la = {}, {}, {}, {}
    for t in T:
        ii_0_la[t], ii_f_la[t], purch_la[t], back_la[t] = get_lookahead_decisions(episode, t, la_decisions, range(t,t+tws[episode,t]), K, O_k, S, ii_0)

    ''' First chart'''
    initi = [sum(ii_0[t][k,o] for k in K for o in range(1,O_k[k]+1)) for t in T]
    repl = [sum(purch[t][i,k] for i in M for k in K) for t in T]
    backo = [sum(back[t][k] for k in K) for t in T]
    # Past and today's realized decisions
    ub1 = max([initi[t]+repl[t]+backo[t] for t in T])
    initi = {(tau,t,s):sum(ii_0_la[tau][t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    repl = {(tau,t,s):sum(purch_la[tau][t][s][i,k] for k in K for i in M_kt[k,t]) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    backo = {(tau,t,s):sum(back_la[tau][t][s][k] for k in K) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    # Future lookahead decisions
    ub1 = max(ub1,*[initi[tau,t,s]+repl[tau,t,s] for tau in T for t in range(tau+1,tau+ tws[episode,tau]) for s in S])
    # Today lookahead decisions
    ub1 = max(ub1,*[initi[tau,tau,s]+repl[tau,tau,s]+backo[tau,tau,s] for tau in T for s in S])
    ub1 = (int(ub1/50)+1)*50

    ''' Second chart '''
    # Max x value
    back_li = [sum(back_la[tau][tau][s][k] for k in K) for s in S for tau in T]
    ub31 = max(back_li)
    ub31 = (int(ub31/10)+1)*10
    # Max value for y axis
    ub32 = 0
    for tau in T:
        hist_b = plt.hist([sum(back_la[tau][tau][s][k] for k in K) for s in S])[0]
        for i in hist_b:
            if i > ub32:
                ub32 = i
    plt.close()

    ''' Fourth chart'''
    purcha = [sum(p[t][i,k]*purch[t][i,k] for i in M for k in K) for t in T]
    hold = [sum(h[t][k]*ii_f[t][k,o] for k in K for o in range(O_k[k]+1)) for t in T]
    backo = [sum(g*back[t][k] for k in K) for t in T]
    routi = [sum(c[rout[t][i][j],rout[t][i][j+1]] for i in range(len(rout[t])) for j in range(len(rout[t][i])-1)) for t in T]
    # Past and today's realized decisions
    ub4 = max([purcha[t]+hold[t]+backo[t]+routi[t] for t in T])
    purch = {(tau,t,s):sum(p[t][i,k]*purch_la[tau][t][s][i,k] for k in K for i in M_kt[k,t]) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    hold = {(tau,t,s):sum(h[t][k]*ii_f_la[tau][t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    backo = {(tau,t,s):sum(g*back_la[tau][t][s][k] for k in K) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    # Today's and future lookahead decisions
    ub4 = max(ub4,*[purch[tau,t,s]+hold[tau,t,s]+backo[tau,t,s] for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S])
    ub4 = (int(ub4/1e4)+1)*1e4

    return [ub1,(ub31,ub32),ub4]

def append_routes(rout):
    sups = []
    for i in rout:
        sups += i[1:-1]
    sups = list(set(sups))

    return sups

def quantity_boxplot(ax, suppliers, q_s, K, S, colors, ind):
    if len(suppliers) == 0:
        sup = 1
    else:
        sup = len(suppliers)

    boxi = ax.boxplot([sum(q_s[s][i,k] for i in suppliers for k in K)/sup for s in S],positions=[0.75+1.5*ind],widths=[0.3],vert=True,patch_artist=True,flierprops = dict(markerfacecolor=colors["box_avail"][1], markeredgecolor=colors["box_avail"][0]))
    
    return boxi

def price_boxplot(ax, suppliers, p, day, K, colors, ind):
    price_list = {i:[p[day][i,k] for k in K if p[day][i,k] < 1e3] for i in suppliers}
    boxi = ax.boxplot([sum(price_list[i])/len(price_list[i]) for i in suppliers],positions=[1.25+1.5*(ind)],widths=[0.3],vert=True,patch_artist=True,flierprops = dict(markerfacecolor=colors["box_prices"][1], markeredgecolor=colors["box_prices"][0]))
    return boxi

def quantities_chart(ax, day, M, K, T, S, M_kt, O_k, d, hist_T, ii_0, purch, back, la_T, ii_0_la, purch_la, back_la, ub1, colors, conf_level, stochastic=True):
    # Realized decisions
    for t in hist_T:
        initi = sum(ii_0[t][k,o] for k in K for o in range(1,O_k[k]+1))
        repl = sum(purch[t][i,k] for i in M for k in K)
        backo = sum(back[t][k] for k in K)
        if t == day:
            x_adj = 0.2
            wid = 0.4
        else:
            x_adj = 0
            wid = 0.8
        ax.bar(x=t-x_adj, height=initi, color=colors["hold"][0],width=wid)
        ax.bar(x=t-x_adj, height=repl, bottom=initi, color=colors["purch"][0],width=wid)
        ax.bar(x=t-x_adj, height=backo, bottom=initi+repl, color=colors["back"][0], width=wid)

    if stochastic:
        # Look-ahead decisions
        for t in la_T:
            initi = [sum(ii_0_la[t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for s in S]
            repl = [sum(purch_la[t][s][i,k] for k in K for i in M_kt[k,t]) for s in S]
            backo = [sum(back_la[t][s][k] for k in K) for s in S]
            if t == day:
                tq = [initi[s]+repl[s]+backo[s] for s in S]
                cin = st.t.interval(confidence=conf_level, df=len(tq)-1, loc=sum(tq)/len(tq), scale=st.sem(tq)) 
                ax.axvline(x=t+0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
                bot_back = (sum(initi)+sum(repl))/len(S)
                x_adj = 0.2
            else:
                tq = [initi[s]+repl[s] for s in S]
                cin = st.t.interval(confidence=conf_level, df=len(tq)-1, loc=sum(tq)/len(tq), scale=st.sem(tq)) 
                ax.axvline(x=t-0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
                cin = st.t.interval(confidence=conf_level, df=len(backo)-1, loc=sum(backo)/len(backo), scale=st.sem(backo))
                ax.axvline(x=t+0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
                bot_back = 0
                x_adj = -0.2
            ax.bar(x=t+x_adj, height=sum(initi)/len(S), color=colors["hold"][0], width=0.4, alpha=0.5)
            ax.bar(x=t+x_adj, height=sum(repl)/len(S), bottom=sum(initi)/len(S), color=colors["purch"][0], width=0.4, alpha = 0.5)
            ax.bar(x=t+0.2, height=sum(backo)/len(S), bottom=bot_back, color=colors["back"][0], width=0.4, alpha=0.5)
        
    # Historical realized demand
    ax.plot([t for t in hist_T],[sum(d[t][k] for k in K) for t in hist_T],linestyle="-",marker="*",markersize=12,color="black")
    
    # Chart config
    ax.set_xlim(-0.5,len(T)-0.5)
    ax.set_ylim(0,ub1)
    ax.set_xlabel("Time period")
    ax.set_ylabel("Quantity")
    ax.bar(x=day,height=0,color=colors["back"][0],label="Backorders")
    ax.bar(x=day,height=0,color=colors["purch"][0],label="Replenishment")
    ax.bar(x=day,height=0,color=colors["hold"][0],label="Initial Inv. Level")
    ax.plot(day+len(T),0,color="black",linestyle="-",marker="*",markersize=9,label="Demand")
    ax.legend(loc="upper right",ncol=2)

    return ax

def backorders_hist_chart(ax, day, K, S, back_la, max_vals, colors):

    ub31,ub32 = max_vals
    # Backorders histogram
    ax.hist([sum(back_la[day][s][k] for k in K) for s in S], color=colors["back"][1], edgecolor=colors["back"][0], alpha = 0.5, density = False, label="Backorders")
    
    # Chart config
    ticks = [i for i in range(0,int(ub31+1),int(ub31/10))]
    ax.set_xticks(ticks=ticks)
    ax.set_xlabel("Units")
    ax.set_xlim(-5,ub31+5)
    ax.set_ylim(0,ub32+1)
    ax.legend(loc="upper center",ncol=2)

    return ax

def quantity_price_box_chart(ax, day, M, K, S, p, q_s, rout, colors):
    # Availability boxplots
    suppliers_visited = {"all":list(M), "st":append_routes(rout[day])}#, "myo":append_routes(rout_myopic[day])}

    box1 = quantity_boxplot(ax,suppliers_visited["all"],q_s,K,S,colors,0)
    box2 = quantity_boxplot(ax,suppliers_visited["st"],q_s,K,S,colors,1)
    #box3 = quantity_boxplot(ax3,suppliers_visited["myo"],q_s,K,S,colors,2)
    for boxi in [box1,box2]:#,box3]:
        for item in ['boxes', 'whiskers', 'fliers', 'medians', 'caps']:
            if item not in ['medians','boxes']:
                plt.setp(boxi[item], color = colors["box_avail"][0])
            elif item == "boxes":
                boxi[item][0].set(color = colors["box_avail"][0])
                boxi[item][0].set(facecolor = colors["box_avail"][1])
            else:
                plt.setp(boxi[item], color = colors["box_avail"][0])

    # Prices boxplots
    ax0 = ax.twinx()
    box5 = price_boxplot(ax0,suppliers_visited["all"],p,day,K,colors,0)
    box6 = price_boxplot(ax0,suppliers_visited["st"],p,day,K,colors,1)
    # box7 = price_boxplot(ax31,suppliers_visited["myo"],p,day,K,colors,2)
    for boxi in [box5,box6]:#,box7]:
        for item in ['boxes', 'whiskers', 'fliers', 'medians', 'caps']:
            if item not in ['medians','boxes']:
                plt.setp(boxi[item], color = colors["box_prices"][0])
            elif item == "boxes":
                boxi[item][0].set(color = colors["box_prices"][0])
                boxi[item][0].set(facecolor = colors["box_prices"][1])
            else:
                plt.setp(boxi[item], color = colors["box_prices"][0])

    # Chart config
    ax0.yaxis.set_major_formatter("${x:1.0f}")
    ax0.set_ylabel("Price",rotation=270,labelpad=12)
    
    ax.set_xticks([1,2.5,4])
    ax.set_xticklabels([f"All Suppliers\n{suppliers_visited['all']}",f"Stochastic\n{suppliers_visited['st']}",f"Myopic"])
    ax.set_xlabel("Subsets of Suppliers")
    ax.set_ylabel("Available Quantity")
    ax.set_xlim(0.5,4.5)
    ax.legend(loc="upper right")

    return ax, ax0

def costs_chart(ax, day, M, K, T, S, M_kt, O_k, hist_T, purch, ii_f, back, rout, la_T, purch_la, ii_f_la, back_la, ub4, costs, colors, conf_level, stochastic=True):
    
    c = costs[0]
    h = costs[1]
    p = costs[2]
    g = costs[3]/1e3

    # Realized decisions
    for t in hist_T:
        purcha = sum(p[t][i,k]*purch[t][i,k] for i in M for k in K)
        hold = sum(h[t][k]*ii_f[t][k,o] for k in K for o in range(O_k[k]+1))
        backo = sum(g*back[t][k] for k in K)
        routi = sum(c[rout[t][i][j],rout[t][i][j+1]] for i in range(len(rout[t])) for j in range(len(rout[t][i])-1))
        if t == day:
            x_adj = 0.2
            wid = 0.4
        else:
            x_adj = 0
            wid = 0.8
        ax.bar(x=t-x_adj, height=purcha, color=colors["purch"][0], width=wid)
        ax.bar(x=t-x_adj, height=hold, bottom=purcha, color=colors["hold"][0], width=wid)
        ax.bar(x=t-x_adj, height=backo, bottom=purcha+hold, color=colors["back"][0], width=wid)
        ax.bar(x=t-x_adj, height=routi, bottom=purcha+hold+backo, color=colors["rout"][0], width=wid)

    # Look-ahead decisions
    if stochastic:
        for t in la_T:
            purch = [sum(p[t][i,k]*purch_la[t][s][i,k] for k in K for i in M_kt[k,t]) for s in S]
            hold = [sum(h[t][k]*ii_f_la[t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for s in S]
            backo = [sum(g*back_la[t][s][k] for k in K) for s in S]
            tc = [purch[s]+hold[s]+backo[s] for s in S]
            if t == day:
                x_adj = 0.2
                width = 0.4
            else:
                x_adj = 0
                width = 0.8
            ax.bar(x=t+x_adj, height=sum(purch)/len(S), color=colors["purch"][0], width = width, alpha = 0.5)
            ax.bar(x=t+x_adj, height=sum(hold)/len(S), bottom=sum(purch)/len(S), color=colors["hold"][0], width = width, alpha = 0.5)
            ax.bar(x=t+x_adj, height=sum(backo)/len(S), bottom=(sum(purch)+sum(hold))/len(S), color=colors["back"][0], width = width, alpha = 0.5)
            cin = st.t.interval(confidence=conf_level, df=len(tc)-1, loc=sum(tc)/len(tc), scale=st.sem(tc))
            ax.axvline(x=t+x_adj,ymin=cin[0]/ub4,ymax=cin[1]/ub4,color="black",marker="_",mew=1.5,ms=8)

    # Chart config
    ax.set_xlim(0-0.5,len(T)-0.5)
    ax.set_ylim(0,ub4)
    ax.bar(x=day,height=0,color=colors["rout"][0],label="Routing")
    ax.bar(x=day,height=0,color=colors["back"][0],label="Backorders")
    ax.bar(x=day,height=0,color=colors["hold"][0],label="Holding")
    ax.bar(x=day,height=0,color=colors["purch"][0],label="Purchase")
    ax.legend(loc="upper right")
    ticks = [i for i in range(0,int(ub4+ub4/10),int(ub4/10))]
    ax.set_yticks(ticks=ticks)
    ax.set_yticklabels(["${:,.0f}K".format(int(i/1e3)) for i in ticks])
    ax.set_ylabel("Total Cost")
    ax.set_xlabel("Time period")

    return ax

def total_cost(day, M, K, O_k, costs, purch, ii_f, back, rout):

    c = costs[0]
    h = costs[1]
    p = costs[2]
    g = costs[3]/1e3

    TT = range(day+1)
    purcha = [sum(p[t][i,k]*purch[t][i,k] for i in M for k in K) for t in TT] 
    hold = [sum(h[t][k]*ii_f[t][k,o] for k in K for o in range(O_k[k]+1)) for t in TT]
    backo = [sum(g*back[t][k] for k in K) for t in TT]
    routi = [sum(c[rout[t][i][j],rout[t][i][j+1]] for i in range(len(rout[t])) for j in range(len(rout[t][i])-1)) for t in TT]
    tc = [purcha[t]+hold[t]+backo[t]+routi[t] for t in TT]

    return tc[-1], sum(tc)

def rainbow_text(ax, x, y, strings, colors, fontsize=16, orientation='horizontal'):
    
    t = ax.transData
    canvas = ax.figure.canvas

    for s, c in zip(strings, colors):
        text = ax.text(x, y, s + " ", color=c, transform=t, fontsize=fontsize)
        # Need to draw to update the text position.
        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        t = text.get_transform() + Affine2D().translate(ex.width, 0)
    
    return ax


def visualize_by_day(episode, day, states, real_actions, backorders, la_decisions, realized_dem, q_sample, tws, iterables, costs, conf_level, max_vals, myopic_actions, perished, save=False):
    
    azul = (41/255,122/255,204/255)
    verde = (70/255,145/255,57/255)
    naranja = (235/255,140/255,68/255)
    morado = (99/255,45/255,235/255)
    rosado = (199/255,93/255,169/255)

    colors = {"hold":[naranja,"palegreen"],
                   "back":[azul,"cyan"],
                   "purch":[verde,"palegreen"],
                   "rout":[rosado,"violet"],
                   "dem":[morado,"mediumpurple"],
                   "box_avail":[(91/255,179/255,77/255),(213/255,230/255,123/255)],
                   "box_prices":[(181/255,21/255,0/255),(223/255,159/255,156/255)]}

    M, K, S, M_kt, O_k, T = iterables

    ''' Realized, Historical Decisions '''
    hist_T = range(day+1)
    ii_0, ii_f, rout, purch, back = get_historical_decisions(episode,states,real_actions,backorders, K, hist_T, O_k, perished)

    ''' Realized demand '''
    d = {t:realized_dem[episode,t] for t in hist_T}

    ''' Look-ahead Decisions '''
    la_T = range(day,day+tws[episode,day])
    ii_0_la, ii_f_la, purch_la, back_la = get_lookahead_decisions(episode, day, la_decisions, la_T, K, O_k, S, ii_0)

    ''' Quantity sample paths '''
    q_s = {s:q_sample[episode,day][s] for s in S}

    ''' Myopic algorithm decisions '''
    #rout_myopic = {t:myopic_actions[episode,t][0] for t in T}

    fig = plt.figure(constrained_layout=True, figsize=(26,20))
    gs = GridSpec(3, 6, figure=fig, wspace=0.165, width_ratios=[1, 1, 1.25, 1.25, 1, 1])

    ''' First Row: Quantities '''
    ax1 = fig.add_subplot(gs[0,:3])
    ax1 = quantities_chart(ax1, day, M, K, T, S, M_kt, O_k, d, hist_T, ii_0, purch, back, la_T, ii_0_la, purch_la, back_la, max_vals[0], colors, conf_level)

    ax2 = fig.add_subplot(gs[0,3:])

    ''' Second Row: Costs '''
    ax3 = fig.add_subplot(gs[1,:3])
    ax3 = costs_chart(ax3, day, M, K, T, S, M_kt, O_k, hist_T, purch, ii_f, back, rout, la_T, purch_la, ii_f_la, back_la, max_vals[2], costs, colors, conf_level, stochastic=True)

    ax4 = fig.add_subplot(gs[1,3:])

    ''' Third Row: comparing '''
    
    ax5 = fig.add_subplot(gs[2,:2])
    ax5 = backorders_hist_chart(ax5, day, K, S, back_la, max_vals[1], colors)

    ax6 = fig.add_subplot(gs[2,2:4])
    ax6, ax61 = quantity_price_box_chart(ax6, day, M, K, S, costs[2], q_s, rout, colors)
    
    ax7 = fig.add_subplot(gs[2,4:])
    ax7.set_xticks([])
    ax7.set_yticks([])
    ax7.text(x=0.4, y=4.75, s="Stochastic RH Perished Products:", ha="left", va="center", fontsize=12)
    ax7.text(x=0.4, y=4.35, s=f"Day {day}: {round(sum(perished[episode,day][k] for k in K),2)} units     Running Total: {round(sum(perished[episode,t][k] for k in K for t in range(day+1)),2)} units", fontsize=16)
    
    st_cost, tot_st_cost = total_cost(day, M, K, O_k, costs, purch, ii_f, back, rout)
    #########################################
    # CAMBIAR DESPUÉS
    m_cost, tot_m_cost = total_cost(day, M, K, O_k, costs, purch, ii_f, back, rout)
    # BORRAR DESPUÉS
    m_cost += 1e3; tot_m_cost -= 5e3
    #########################################

    if m_cost < st_cost:
        col1 = "red"
    else:
        col1 = "green"
    if tot_m_cost < tot_st_cost:
        col2 = "red"
    else:
        col2 = "green"

    ax7.text(x=0.4, y= 3.75, s="Stochastic RH Total Cost:", ha="left", va="center", fontsize=12)
    ax7 = rainbow_text(ax = ax7, x=0.4, y=3.35, strings=[f"Day {day}:","${:,.2f}K,".format(st_cost/1e3),"{:+,.2%}".format((st_cost-m_cost)/m_cost), "from myopic"], colors=["black","black",col1,"black"])
    ax7 = rainbow_text(ax = ax7, x=0.4, y=2.95, strings=[f"Running Total:","${:,.2f}K".format(tot_st_cost/1e3),"{:+,.2%}".format((tot_st_cost-tot_m_cost)/tot_m_cost), "from myopic"], colors=["black","black",col2,"black"])

    ax7.axis("off")
    ax7.set_ylim(0,5)
    ax7.set_xlim(0,5)
    # Metrics
    

    if save:
        plt.savefig(f'./RH shots/Rolling_{day}.png', dpi=300)
        plt.close()


In [ ]:
max_vals = charts_max_axis_values(0, states, real_actions, backorders, la_decisions, tws, iterables, costs, perished)
myopic_actions = [0]
visualize_by_day(0, 6, states, real_actions, backorders, la_decisions, realized_dem, q_sample, tws, iterables, costs, 0.95, max_vals, myopic_actions, perished)

In [ ]:
def display_policy_gif():
    
    for t in range(env_config["T"]):
        visualize_by_day(0, t, states, real_actions, backorders, la_decisions, realized_dem, q_sample, tws, iterables, costs, 0.9, max_vals, myopic_actions, perished, True)

    images = [imageio.imread(f"./RH shots/Rolling_{day}.png") for day in range(env_config["T"])]
    imageio.mimsave("./RH shots/RH.gif", images,duration=1)
    
    return Image(open("./RH shots/RH.gif",'rb').read(),width=600)
    
display_policy_gif()

In [ ]:
print(realized_q[0,3])
print(q_sample[0,3][0])